In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader ,Dataset
from transformers import AutoTokenizer,BertForQuestionAnswering,AutoModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
from transformers import AutoTokenizer,BertForQuestionAnswering,AutoModel
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [3]:
data=pd.read_csv('/kaggle/input/question-pairs-dataset/questions.csv')

In [4]:
questions = ["What is the capital of France?", "Who wrote the play 'Romeo and Juliet'?", "When was the Eiffel Tower built?"]
answers = ["The capital of France is Paris.", "The play 'Romeo and Juliet' was written by William Shakespeare.", "The Eiffel Tower was built in 1889."]

In [3]:
from transformers import DataCollatorWithPadding

In [6]:
w=dict()

In [7]:
data.dropna(inplace=True)

In [8]:
class dataset(Dataset):
    def __init__(self,x,y,z):
        self.x=list(x)
        self.y=list(y)
        out=  tokenizer(self.x,self.y, truncation=True, max_length=30,padding='max_length', return_tensors="pt")
        self.keys=out.keys
        self.out= list(out.items())

        self.z=z
    def __getitem__ (self,idx):
        r={key:value[idx] for key ,value in self.out }

        return r,self.z[idx]
    def __len__(self):
        return len(self.x)

In [12]:
torch.set_default_device('cpu')


In [13]:
data=data.reset_index()

In [14]:
w=dataset(data.iloc[:,-2],data.iloc[:,-3],data.iloc[:,-1])
train=DataLoader(w,batch_size=32,shuffle=True, generator=torch.Generator(device='cpu'))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 w=dataset(data.iloc[:,-2],data.iloc[:,-3],data.iloc[:,-1])                                   │
│   2 train=DataLoader(w,batch_size=32,shuffle=True, generator=torch.Generator(device='cpu'))      │
│   3                                                                                              │
│                                                                                                  │
│ in __init__:5                                                                                    │
│                                                                                                  │
│    2 │   def __init__(self,x,y,z):                                                               │
│    3 │   │   self.x=list(x)                                                                      │
│    4 │   │   self.y=list(y)                                                                      │
│ ❱  5 │   │   out=  tokenizer(self.x,self.y, truncation=True, max_length=30,padding='max_lengt    │
│    6 │   │   self.keys=out.keys                                                                  │
│    7 │   │   self.out= list(out.items())                                                         │
│    8                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2561 in __call__ │
│                                                                                                  │
│   2558 │   │   │   # input mode in this case.                                                    │
│   2559 │   │   │   if not self._in_target_context_manager:                                       │
│   2560 │   │   │   │   self._switch_to_input_mode()                                              │
│ ❱ 2561 │   │   │   encodings = self._call_one(text=text, text_pair=text_pair, **all_kwargs)      │
│   2562 │   │   if text_target is not None:                                                       │
│   2563 │   │   │   self._switch_to_target_mode()                                                 │
│   2564 │   │   │   target_encodings = self._call_one(text=text_target, text_pair=text_pair_targ  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2647 in          │
│ _call_one                                                                                        │
│                                                                                                  │
│   2644 │   │   │   │   │   f" {len(text_pair)}."                                                 │
│   2645 │   │   │   │   )                                                                         │
│   2646 │   │   │   batch_text_or_text_pairs = list(zip(text, text_pair)) if text_pair is not No  │
│ ❱ 2647 │   │   │   return self.batch_encode_plus(                                                │
│   2648 │   │   │   │   batch_text_or_text_pairs=batch_text_or_text_pairs,                        │
│   2649 │   │   │   │   add_special_tokens=add_special_tokens,                                    │
│   2650 │   │   │   │   padding=padding,                                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2838 in          │
│ batch_encode_plus                                          

In [9]:
from transformers import BertTokenizer, BertModel


In [10]:
class bert_compare(torch.nn.Module):
    def __init__ (self):
        super(bert_compare,self).__init__()
        self.bert=BertModel.from_pretrained("bert-base-uncased")
        
        self.Linear=torch.nn.Linear(768,30 )
        self.elu=torch.nn.ELU()
        self.Linear2=torch.nn.Linear(280 ,1 )
        self.cnn1=torch.nn.Conv1d(768,256,kernel_size=2)
        self.cnn2=torch.nn.Conv1d(256,10,kernel_size=2)

        self.relu=torch.nn.ReLU()
    def forward(self,x):
        x=self.bert(**x).last_hidden_state
        x=x.permute(0,2,1)
        x=self.cnn1(x)
        x=self.relu(x)
        x=self.cnn2(x)
        x=torch.nn.Flatten()(x)
        x=self.Linear2(x)
        return x

In [11]:
model=bert_compare()
optim=torch.optim.AdamW(model.parameters(),lr=5e-5)
loss=torch.nn.BCEWithLogitsLoss()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
def tok(x,y):
        out=tokenizer(x,y, truncation=True, max_length=30,padding='max_length', return_tensors="pt")
        out={key:value for key,value in out.items()}
        return out
h=tok('my name is mohamed','what is your name')    
model(h)

tensor([[-0.0260]], grad_fn=<AddmmBackward0>)

In [19]:
from  tqdm import tqdm

In [20]:
model.train()

bert_compare(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [ ]:
model.bert.train()

In [26]:
for i in range(10):
  loop=tqdm(train,leave=True)

  for x,y in loop :
        optim.zero_grad()
        out=model(x)
        l=loss(out,y.unsqueeze(-1).float())
        l.backward()
        optim.step()
        loop.set_description(f'Epoch {i}')
        loop.set_postfix(loss=l.item())
  torch.save(model,f'model{i}.pth')
  torch.save(model,f'model{i}.pt')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 for i in range(10):                                                                         │
│ ❱  2   loop=tqdm(train,leave=True)                                                               │
│    3                                                                                             │
│    4   for x,y in loop :                                                                         │
│    5 │   │   optim.zero_grad()                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'train' is not defined

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
model=torch.load('/kaggle/input/notebookb537c02774/model9.pth',map_location=torch.device('cpu'))

In [15]:
from transformers import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="mohamedemam/bert_sentaces_similarty", tokenizer=tokenizer, hub_model_id="mohamedemam/bert_sentaces_similarty"
)

Cloning https://huggingface.co/mohamedemam/bert_sentaces_similarty into local empty directory.


In [18]:
from transformers import PreTrainedModel
from timm.models.resnet import BasicBlock, Bottleneck, ResNet


In [25]:
from transformers.configuration_resnet import ResnetConfig


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 from transformers.configuration_resnet import ResnetConfig                                   │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'transformers.configuration_resnet'

In [22]:
class bert_compare(PreTrainedModel):
    def __init__ (self,config):
        super().__init__(config)
        self.bert=BertModel.from_pretrained("bert-base-uncased")
        
        self.Linear=torch.nn.Linear(768,30 )
        self.elu=torch.nn.ELU()
        self.Linear2=torch.nn.Linear(280 ,1 )
        self.cnn1=torch.nn.Conv1d(768,256,kernel_size=2)
        self.cnn2=torch.nn.Conv1d(256,10,kernel_size=2)

        self.relu=torch.nn.ReLU()
    def forward(self,x):
        x=self.bert(**x).last_hidden_state
        x=x.permute(0,2,1)
        x=self.cnn1(x)
        x=self.relu(x)
        x=self.cnn2(x)
        x=torch.nn.Flatten()(x)
        x=self.Linear2(x)
        return x

In [41]:
from transformers import PretrainedConfig
from typing import List


class ResnetConfig(PretrainedConfig):
    model_type = "resnet"

    def __init__(
        self,
        **kwargs,
    ):

        super().__init__(**kwargs)

In [42]:
c=ResnetConfig()
model=bert_compare(c)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [43]:
model2=torch.load("/kaggle/input/notebookb537c02774/model9.pth",map_location=torch.device('cpu'))

In [44]:
  model.bert=        model2.bert      
  model.Linear =     model2.Linear
  model.elu     =  model2.elu
  model.Linear2  =     model2.Linear2
  model.cnn1      = model2.cnn1
  model.cnn2     = model2.cnn2
  model.relu   = model2.relu

In [45]:
word=['my name is mohamed ', "How do I read and find my YouTube comments?" ,"How can I see all my Youtube comments?","How can Internet speed be increased by hacking through DNS?","What is the step by step guide to invest in share market in india?","where is capital of egypt?",'when did you born ','what is your name',"what is capital of egypt",'how old are you']

In [46]:
def tok(x,y):
        out=tokenizer(x,y, truncation=True, max_length=30,padding='max_length', return_tensors="pt")
        out={key:value for key,value in out.items()}
        return out
for i in range(9):
        r=torch.randint(len(word),size=(1,))
        r2=torch.randint(len(word),size=(1,))
        h=tok(word[r],word[r2])    
        e=model(h)
        ans= 'the same' if  int(torch.sigmoid( e)>=.5) else 'not the same'
        print (f'{word[r]} is {ans} {word[r2]}'  )

In [47]:

        h=tok("what is capital of egypt","when is  capital of egypt")    
        e=model(h)
        ans= 'the same' if  int(torch.sigmoid( e)>=.5) else 'not the same'
        print (f' {ans} '  )

In [50]:
model.push_to_hub("mohamedemam/bert_sentaces_similarty")

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mohamedemam/bert_sentaces_similarty/commit/88e8c9dbba52459df6dc3ea024226e59139a40d1', commit_message='Upload bert_compare', commit_description='', oid='88e8c9dbba52459df6dc3ea024226e59139a40d1', pr_url=None, pr_revision=None, pr_num=None)

In [51]:
tokenizer.push_to_hub("mohamedemam/bert_sentaces_similarty")

CommitInfo(commit_url='https://huggingface.co/mohamedemam/bert_sentaces_similarty/commit/10622cafd9ce8dda9cb8592691742436a05ad3d4', commit_message='Upload tokenizer', commit_description='', oid='10622cafd9ce8dda9cb8592691742436a05ad3d4', pr_url=None, pr_revision=None, pr_num=None)